# CVAE for MNIST  

The purpose of this demo is to help you learn about variational autoencoder. The algorithm is being implemented is from the paper "Learning Structured Output Representation using Deep Conditional Generative Models" by 
Kihyuk Sohn, Honglak Lee, Xinchen Yan (https://openreview.net/forum?id=rJWXGDWd-H).

Followed the logic in https://github.com/AntixK/PyTorch-VAE/blob/master/models/cvae.py.

You are free to change model acrhitecture, or any part of the logic. 

If you have any suggestions or find errors, please, don't be hesitate to text me at jayanta.jayantamukherjee@gmail.com


In [1]:
import torch
from torch import nn
from torch import Tensor
from torch.nn import functional as F


### Conditional VAE Models 

In [2]:
class ConditionalVAE(torch.nn.Module):

    def __init__(self,
                 in_channels: int,
                 num_classes: int,
                 latent_dim: int,
                 hidden_dims: [] = None,
                 img_size:int = 64,
                 **kwargs) -> None:
        super(ConditionalVAE, self).__init__()
        self.latent_dim = latent_dim
        self.img_size = img_size

        self.embed_class = nn.Linear(num_classes, img_size * img_size)
        self.embed_data = nn.Conv2d(in_channels, in_channels, kernel_size=1)

        modules = []
        if hidden_dims is None:
            hidden_dims = [32, 64, 128, 256, 512]

        in_channels += 1 # To account for the extra label channel
        # Build Encoder
        for h_dim in hidden_dims:
            modules.append(
                nn.Sequential(
                    nn.Conv2d(in_channels, out_channels=h_dim,
                              kernel_size= 3, stride= 2, padding  = 1),
                    nn.BatchNorm2d(h_dim),
                    nn.LeakyReLU())
            )
            in_channels = h_dim

        self.encoder = nn.Sequential(*modules)
        self.fc_mu = nn.Linear(hidden_dims[-1]*4, latent_dim)
        self.fc_var = nn.Linear(hidden_dims[-1]*4, latent_dim)


        # Build Decoder
        modules = []

        self.decoder_input = nn.Linear(latent_dim + num_classes, hidden_dims[-1] * 4)

        hidden_dims.reverse()

        for i in range(len(hidden_dims) - 1):
            modules.append(
                nn.Sequential(
                    nn.ConvTranspose2d(hidden_dims[i],
                                       hidden_dims[i + 1],
                                       kernel_size=3,
                                       stride = 2,
                                       padding=1,
                                       output_padding=1),
                    nn.BatchNorm2d(hidden_dims[i + 1]),
                    nn.LeakyReLU())
            )



        self.decoder = nn.Sequential(*modules)

        self.final_layer = nn.Sequential(
                            nn.ConvTranspose2d(hidden_dims[-1],
                                               hidden_dims[-1],
                                               kernel_size=3,
                                               stride=2,
                                               padding=1,
                                               output_padding=1),
                            nn.BatchNorm2d(hidden_dims[-1]),
                            nn.LeakyReLU(),
                            nn.Conv2d(hidden_dims[-1], out_channels= 3,
                                      kernel_size= 3, padding= 1),
                            nn.Tanh())

    def encode(self, input: Tensor):
        """
        Encodes the input by passing through the encoder network
        and returns the latent codes.
        :param input: (Tensor) Input tensor to encoder [N x C x H x W]
        :return: (Tensor) List of latent codes
        """
        result = self.encoder(input)
        result = torch.flatten(result, start_dim=1)

        # Split the result into mu and var components
        # of the latent Gaussian distribution
        mu = self.fc_mu(result)
        log_var = self.fc_var(result)

        return [mu, log_var]

    def decode(self, z: Tensor) -> Tensor:
        result = self.decoder_input(z)
        result = result.view(-1, 512, 2, 2)
        result = self.decoder(result)
        result = self.final_layer(result)
        return result

    def reparameterize(self, mu: Tensor, logvar: Tensor) -> Tensor:
        """
        Will a single z be enough to compute the expectation
        for the loss??
        :param mu: (Tensor) Mean of the latent Gaussian
        :param logvar: (Tensor) Standard deviation of the latent Gaussian
        :return:
        """
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps * std + mu

    def forward(self, input: Tensor, **kwargs):
        y = kwargs['labels'].float()
        embedded_class = self.embed_class(y)
        embedded_class = embedded_class.view(-1, self.img_size, self.img_size).unsqueeze(1)
        embedded_input = self.embed_data(input)

        x = torch.cat([embedded_input, embedded_class], dim = 1)
        mu, log_var = self.encode(x)

        z = self.reparameterize(mu, log_var)

        z = torch.cat([z, y], dim = 1)
        return  [self.decode(z), input, mu, log_var]

    def loss_function(self,
                      *args,
                      **kwargs) -> dict:
        recons = args[0]
        input = args[1]
        mu = args[2]
        log_var = args[3]

        kld_weight = kwargs['M_N']  # Account for the minibatch samples from the dataset
        recons_loss =F.mse_loss(recons, input)

        kld_loss = torch.mean(-0.5 * torch.sum(1 + log_var - mu ** 2 - log_var.exp(), dim = 1), dim = 0)

        loss = recons_loss + kld_weight * kld_loss
        return {'loss': loss, 'Reconstruction_Loss':recons_loss, 'KLD':-kld_loss}

    def sample(self,
               num_samples:int,
               current_device: int,
               **kwargs) -> Tensor:
        """
        Samples from the latent space and return the corresponding
        image space map.
        :param num_samples: (Int) Number of samples
        :param current_device: (Int) Device to run the model
        :return: (Tensor)
        """
        y = kwargs['labels'].float()
        z = torch.randn(num_samples,
                        self.latent_dim)

        z = z.to(current_device)

        z = torch.cat([z, y], dim=1)
        samples = self.decode(z)
        return samples

    def generate(self, x: Tensor, **kwargs) -> Tensor:
        """
        Given an input image x, returns the reconstructed image
        :param x: (Tensor) [B x C x H x W]
        :return: (Tensor) [B x C x H x W]
        """

        return self.forward(x, **kwargs)[0]

### VAE

In [3]:
import yaml
import argparse
import numpy as np

from VAEXperiment import VAEXperiment
from pytorch_lightning import Trainer
from pytorch_lightning.logging import TestTubeLogger


model = ConditionalVAE(in_channels = 3,
                       num_classes = 40,
                       latent_dim = 128)

exp_params = {
  "dataset": "celeba",
  "data_path": "../../shared/Data/",
  "img_size": 64,
  "batch_size": 144,
  "LR": 0.005,
  "weight_decay": 0.0,
  "scheduler_gamma": 0.95
}

trainer_params = {
  "gpus": 1,
  "max_nb_epochs": 50,
  "max_epochs": 50
}

experiment = VAEXperiment(model,
                          exp_params)

tt_logger = TestTubeLogger(
    save_dir="logs/",
    name="ConditionalVAE",
    debug=False,
    create_git_tag=False
)

runner = Trainer(default_save_path=tt_logger.save_dir,
                 min_nb_epochs=1,
                 logger=tt_logger,
                 log_save_interval=100,
                 train_percent_check=1.,
                 val_percent_check=1.,
                 num_sanity_val_steps=5,
                 early_stop_callback = False,
                 gpus = 1,
                 max_nb_epochs = 50,
                 max_epochs = 50)

print("======= Training ConditionalVAE =======")
runner.fit(experiment)

INFO:root:gpu available: True, used: True
INFO:root:VISIBLE GPUS: 0


======= Training ConditionalVAE =======
Files already downloaded and verified


INFO:root:
                   Name             Type Params
0                 model   ConditionalVAE    4 M
1     model.embed_class           Linear  167 K
2      model.embed_data           Conv2d   12  
3         model.encoder       Sequential    1 M
4       model.encoder.0       Sequential    1 K
5     model.encoder.0.0           Conv2d    1 K
6     model.encoder.0.1      BatchNorm2d   64  
7     model.encoder.0.2        LeakyReLU    0  
8       model.encoder.1       Sequential   18 K
9     model.encoder.1.0           Conv2d   18 K
10    model.encoder.1.1      BatchNorm2d  128  
11    model.encoder.1.2        LeakyReLU    0  
12      model.encoder.2       Sequential   74 K
13    model.encoder.2.0           Conv2d   73 K
14    model.encoder.2.1      BatchNorm2d  256  
15    model.encoder.2.2        LeakyReLU    0  
16      model.encoder.3       Sequential  295 K
17    model.encoder.3.0           Conv2d  295 K
18    model.encoder.3.1      BatchNorm2d  512  
19    model.encoder.3.2      

Epoch 1:   0%|          | 0/1268 [00:00<?, ?batch/s]

Epoch 2:   0%|          | 0/1268 [00:00<04:18,  4.90batch/s, batch_idx=1129, gpu=0, loss=0.090, v_num=0]   

...

Epoch 48:  90%|█████████ | 1147/1268 [05:22<00:20,  6.02batch/s, batch_idx=1129, gpu=0, loss=0.070, v_num=0]


Epoch 49:  91%|█████████ | 1157/1268 [05:23<00:20,  5.44batch/s, batch_idx=1129, gpu=0, loss=0.071, v_num=0]


Epoch 50:  92%|█████████▏| 1167/1268 [05:29<00:15,  6.32batch/s, batch_idx=1129, gpu=0, loss=0.071, v_num=0]


Epoch 50: 100%|██████████| 1268/1268 [05:46<00:00,  3.66batch/s, batch_idx=1129, gpu=0, loss=0.071, v_num=0]


1